## IBM Data Science Capstone 
Week 3 submission 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Here, i am  downloading the data in first two line of code.
In next 3 lines, i have initialised the variable that we will be using in this cell
After that, I have assinged the html file data to soup variable and with the hel of nested for loop, I have assigned it to variable that we defined earlier 

In [2]:
!wget -q -O 'sample.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')
postcode =[]
borough=[]
neighbourhood=[]
with open('sample.html') as Sampledata:
    soup = BeautifulSoup(Sampledata , 'lxml')
table = soup.find('table')
data=[]
for row in table.find_all('tr'):
    for cell in row.find_all('td'):
        data.append(cell.text)
    if(len(data)>1):
        postcode.append(data[0])
        borough.append(data[1])
        neighbourhood.append(data[2].rstrip('\n'))
    data=[]

Data downloaded!


As all the data are stored in variables, i have now created a dataframe to store the data.
Then removed Not assigned values from Borough and changed the Not assignes value of neighbourhood into same as that of corrosponding borough.

In [3]:
canada_df=pd.DataFrame({"Postal Code": postcode, "Borough" : borough, "Neighbourhood" : neighbourhood})
canada_df['Borough'].replace('Not assigned',np.nan,inplace=True)
mod_canada_df= canada_df.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))
for index, row in mod_canada_df.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
print(mod_canada_df.head())

  Postal Code      Borough                         Neighbourhood
0         M1B  Scarborough                         Rouge,Malvern
1         M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2         M1E  Scarborough       Guildwood,Morningside,West Hill
3         M1G  Scarborough                                Woburn
4         M1H  Scarborough                             Cedarbrae


Final Shape of the data frame as output

In [4]:
print(mod_canada_df.shape)

(103, 3)


Now we are going to insert latitude and longitude data into the our dataframe and then we will join both the data frame

In [5]:
map_data=pd.read_csv("https://cocl.us/Geospatial_data")
map_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
final_table = mod_canada_df.set_index('Postal Code').join(map_data.set_index('Postal Code'))
final_table = final_table.reset_index(drop=False)
final_table.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
